<a href="https://colab.research.google.com/github/natalia-va/makeTest2/blob/master/Neural_recomend_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seaborn
!pip install lenskit
!pip install pandas
!pip install torch
!pip install torchviz

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torchviz import make_dot

from lenskit import batch, util
from lenskit import crossfold as xf
from torch.autograd import Variable


import numpy as np

import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/HaykInanc/move_data.git 

fatal: destination path 'move_data' already exists and is not an empty directory.


In [ ]:
ratings = pd.read_csv('/content/move_data/ratings.csv') #ПользовательId-фильмId-рейтинг
movies = pd.read_csv('/content/move_data/movies.csv') #ФильмId-название-жанры

In [ ]:
n_users  = ratings.userId.nunique() #Число уникальных пользователей
n_movies = ratings.movieId.nunique() #Число уникальных фильмов

n_fich = 50 #размерность в embed

In [ ]:
#Поменяем userId и movieId в базовых таблицах на последовательные, начинающиеся с 0.
#
def data_preparation(movies, ratings):
  
    uniq_users = ratings.userId.unique()
    uniq_movies = ratings.movieId.unique()

    #Таблица movie избыточна, удалим далее неоцененные фильмы
    id_rated_films = uniq_movies
    id_not_rated_films = [x for x in movies['movieId'] if x not in id_rated_films]

    if len(id_not_rated_films) !=0:
        id_fields_not_rated_films_in_movie = [movies[movies['movieId'] == i].index[0] for i in id_not_rated_films]
        movies = movies.drop(id_fields_not_rated_films_in_movie)

    #Словари соответствия старого индекса новому
    useridtoidx = {o:i for i,o in enumerate(uniq_users)} 
    movieidtoidx= {o:i for i,o in enumerate(uniq_movies)}

    ratings.userId = ratings.userId.apply(lambda x:useridtoidx[x])
    ratings.movieId = ratings.movieId.apply(lambda x:movieidtoidx[x])
    movies.movieId = movies.movieId.apply(lambda x:movieidtoidx[x])

    return movies, ratings
 

In [ ]:
movies, ratings = data_preparation(movies, ratings)
ratings = ratings.rename(columns={'userId': 'user', 'movieId': 'movie'})
ratings = ratings.rename(columns={'movieId': 'movie'})

In [ ]:
#Разделение на тренировочную и обучающую выборку
data = []

for train, test in xf.partition_users(ratings, 1, xf.SampleFrac(0.2)):
    data.append(train)
    data.append(test)

In [ ]:
#Функция ошибки 
def my_RMSE(y_true, y_pred):
        return torch.sqrt(torch.mean(torch.square(y_pred - y_true), axis=-1))

In [ ]:
class model_rec_system(nn.Module):

    def __init__(self, n_users, n_movies, n_fich):
        super().__init__()
        self.eu = nn.Embedding(n_users, n_fich)
        self.eu.weight.data.uniform_(-0.01,0.01)
       
        self.em = nn.Embedding(n_movies, n_fich)
        self.em.weight.data.uniform_(-0.01,0.01)

        # layer 1 fully connected 150 units
        self.lin1 = nn.Linear(n_fich*2, 150)
        
        # layer 2 fully connected 50 units
        self.lin2 = nn.Linear(150, 50)
        
        # layer 3 fully connected 1 unit (output)
        self.lin3 = nn.Linear(50, 1)
        
        # dropouts
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.4)
        self.drop3 = nn.Dropout(0.25)
        
    def forward(self, data):

        users = data[0]
        movies = data[1]
        x = torch.cat([self.eu(users),self.em(movies)], dim=1)
        x = self.drop1(x)
        x = self.drop2(F.relu(self.lin1(x)))
        x = self.drop3(F.relu(self.lin2(x)))
        
        # output
        x = F.sigmoid(self.lin3(x)) * (max_rating-min_rating+1) + min_rating

        return x

#ограничивающие оценку значения
max_rating = 5 
min_rating = 0

device = torch.device('cuda') if torch.cuda.is_available()  else torch.device('cpu')
net = model_rec_system(n_users, n_movies, n_fich)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
losses = []


In [ ]:
X_train = [torch.tensor(data[0]['user'].values.tolist()), torch.tensor(data[0]['movie'].values.tolist())]
Y_train = torch.tensor(data[0]['rating'].values.tolist()).view(-1,1)

X_test = [data[1]['user'].values, data[1]['movie'].values]
Y_test = data[1]['rating'].values


In [ ]:
criterion = nn.MSELoss()
n_epoch = 20
for epoch in range(n_epoch):
   y_pred = net(X_train)
   loss = criterion(Y_train, y_pred)

   print('epoch: ', epoch,' loss: ', loss.item())
   optimizer.zero_grad()

   loss.backward()
   optimizer.step()
      


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch:  0  loss:  0.7825955152511597
epoch:  1  loss:  0.7788111567497253
epoch:  2  loss:  0.7740965485572815
epoch:  3  loss:  0.7692669034004211
epoch:  4  loss:  0.7668476104736328
epoch:  5  loss:  0.7632676362991333
epoch:  6  loss:  0.7601438164710999
epoch:  7  loss:  0.7563274502754211
epoch:  8  loss:  0.7527014017105103
epoch:  9  loss:  0.7461792230606079
epoch:  10  loss:  0.743732213973999
epoch:  11  loss:  0.7382072806358337
epoch:  12  loss:  0.73570317029953
epoch:  13  loss:  0.7307327389717102
epoch:  14  loss:  0.7267405986785889
epoch:  15  loss:  0.7221246361732483
epoch:  16  loss:  0.7170436978340149
epoch:  17  loss:  0.7153509855270386
epoch:  18  loss:  0.7118858098983765
epoch:  19  loss:  0.7088191509246826


In [ ]:
a = [torch.tensor([3]), torch.tensor([3])]
print(net(a))